In [2]:
from datasets import load_dataset
import numpy as np

raw_dataset = load_dataset("javilonso/TEST_mex_data_title_with_opinion")

Downloading:   0%|          | 0.00/785 [00:00<?, ?B/s]

Using custom data configuration javilonso--TEST_mex_data_title_with_opinion-73c39d43454c2556


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /home/javilonso/.cache/huggingface/datasets/parquet/javilonso--TEST_mex_data_title_with_opinion-73c39d43454c2556/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['Id', 'Title', 'Opinion', 'Title_Opinion'],
        num_rows: 12938
    })
})

In [9]:
raw_dataset["train"][0]

{'Id': '1',
 'Title': 'Súper ..hotel,vacaciones ,lugar y precio !!',
 'Opinion': 'Magnifico lugar,gracias a la Srita marcela Arechiga x su trato y sus costos bajos ,animadores ,comidas y trato genial en el comedor ,bares ,picina ,todos son geniales al muchacho de las toallas .. Volveremos las próximas vacaciones sin dudarlo !\nEn cuanto al lugar es tan bueno ..que los días se hacen cortos .!!!\nBesos y abrazos para todos !!!!',
 'Title_Opinion': 'Súper ..hotel,vacaciones ,lugar y precio !! Magnifico lugar,gracias a la Srita marcela Arechiga x su trato y sus costos bajos ,animadores ,comidas y trato genial en el comedor ,bares ,picina ,todos son geniales al muchacho de las toallas .. Volveremos las próximas vacaciones sin dudarlo !\nEn cuanto al lugar es tan bueno ..que los días se hacen cortos .!!!\nBesos y abrazos para todos !!!!'}

| =================== ATTRACTION =================== |

In [11]:
from transformers import pipeline

text_model_attraction = pipeline("text-classification", model="javilonso/Mex_Rbta_TitleWithOpinion_Attraction")

Downloading:   0%|          | 0.00/865 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at javilonso/Mex_Rbta_TitleWithOpinion_Attraction.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/831k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/497k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [30]:
attraction_predictions = []
def classify_text_attraction(examples):
    elements = examples["Title_Opinion"].copy()
    for item in elements:
        output = text_model_attraction(item)
        predicted_label = output[0]['label']
        
        # Convert to original labels
        if predicted_label == "LABEL_0":
            attraction_predictions.append("Hotel")
        elif predicted_label == "LABEL_1":
            attraction_predictions.append("Restaurant")
        elif predicted_label == "LABEL_2":
            attraction_predictions.append("Attractive")
        else:
            print(predicted_label)

In [31]:
raw_dataset.map(classify_text_attraction, batched=True)
# print(len(attraction_predictions))

  0%|          | 0/13 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['Id', 'Title', 'Opinion', 'Title_Opinion'],
        num_rows: 12938
    })
})

In [37]:
datapd = raw_dataset["train"].to_pandas()
datapd["Attraction"] = attraction_predictions
raw_dataset = Dataset.from_pandas(datapd)

In [42]:
raw_dataset

Dataset({
    features: ['Id', 'Title', 'Opinion', 'Title_Opinion', 'Attraction'],
    num_rows: 12938
})

In [43]:
raw_dataset = Dataset.from_pandas(datapd)
raw_dataset.push_to_hub("javilonso/PRED_TEST_mex_data_title_with_opinion")

The repository already exists: the `private` keyword argument will be ignored.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

| =================== POLARITY =================== |

In [45]:
from transformers import pipeline

text_model_polarity = pipeline("text-classification", model="javilonso/Mex_Rbta_TitleWithOpinion_Augmented_Polarity")

Downloading:   0%|          | 0.00/941 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at javilonso/Mex_Rbta_TitleWithOpinion_Augmented_Polarity.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/831k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/497k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [51]:
polarity_predictions = []
def classify_text_polarity(examples):
    elements = examples["Title_Opinion"].copy()
    for item in elements:
        output = text_model_polarity(item)
        # print(output)
        predicted_label = output[0]['label']
        
        # Convert to original labels
        if predicted_label == "LABEL_0":
            polarity_predictions.append(1)
        elif predicted_label == "LABEL_1":
            polarity_predictions.append(2)
        elif predicted_label == "LABEL_2":
            polarity_predictions.append(3)
        elif predicted_label == "LABEL_3":
            polarity_predictions.append(4)
        elif predicted_label == "LABEL_4":
            polarity_predictions.append(5)
        else:
            print(predicted_label)
            polarity_predictions.append(predicted_label)

In [52]:
raw_dataset.map(classify_text_polarity, batched=True)
# print(len(attraction_predictions))

  0%|          | 0/13 [00:00<?, ?ba/s]

Dataset({
    features: ['Id', 'Title', 'Opinion', 'Title_Opinion', 'Attraction'],
    num_rows: 12938
})

In [55]:
datapd = raw_dataset.to_pandas()
datapd["Polarity"] = polarity_predictions
raw_dataset = Dataset.from_pandas(datapd)
raw_dataset.push_to_hub("javilonso/PRED_TEST_mex_data_title_with_opinion")

The repository already exists: the `private` keyword argument will be ignored.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

In [58]:
datapd = datapd.drop('Title_Opinion', 1)


In [60]:
datapd.to_excel("predictions.xlsx")